# Row-Level Security: Unity Catalog → Lakebase Sync

This notebook demonstrates how to:

1. **Set up row-level access control in Unity Catalog** using dynamic views and `is_account_group_member()`.
2. **Mirror those rules into Lakebase** (PostgreSQL) so the DICOMweb cache respects the same restrictions.
3. **Sync user → group mappings** from the Databricks SCIM API into Lakebase.
4. **Run the sync job** that tags each cached row with `allowed_groups` so PostgreSQL RLS policies can enforce access.

---

### Why is this needed?

The Pixels DICOMweb application uses a **3-tier caching hierarchy**:

| Tier | Backend | Lifetime |
|------|---------|----------|
| 1 | In-memory (`BOTCache`, `InstancePathCache`) | Process lifetime |
| 2 | Lakebase (PostgreSQL) | Persistent across restarts |
| 3 | Unity Catalog SQL Warehouse | Source of truth |

When Unity Catalog **row filters** restrict which DICOM files a user may see, the caches
at Tiers 1 and 2 could bypass those filters — returning cached rows the user should not
have access to.

The solution:
- **Tier 1 (in-memory)** — cache keys include a hash of the user's groups, so different
  users never share entries.
- **Tier 2 (Lakebase)** — each row carries an `allowed_groups` array; PostgreSQL RLS
  policies filter rows based on the user's groups set via `SET LOCAL app.user_groups = ...`.

> **Prerequisites**: A Databricks workspace with Unity Catalog enabled, a Lakebase
> instance provisioned, and the `dbx-pixels` package installed.


## 0. Configuration

Edit the variables below to match your environment.


In [ ]:
# ── Unity Catalog coordinates ──────────────────────────────────────────
CATALOG   = "main"
SCHEMA    = "pixels_solacc"
VOLUME    = "pixels_volume"
TABLE     = "object_catalog"       # the pixels table with DICOM metadata

# Fully qualified table name (used throughout)
UC_TABLE  = f"{CATALOG}.{SCHEMA}.{TABLE}"

# ── Group names ─────────────────────────────────────────────────────────
# These must match your Databricks Account groups.
GROUP_PHI            = "pixels_phi"            # full access to all DICOM data
GROUP_DEIDENTIFIED   = "pixels_deidentified"   # access only to de-identified data

# ── Lakebase ────────────────────────────────────────────────────────────
LAKEBASE_INSTANCE    = "pixels-lakebase"       # name of the Lakebase instance


---
## 1. Unity Catalog — Grant base permissions

Before creating row filters, every group needs `USE CATALOG`, `USE SCHEMA`, `READ VOLUME`,
and `SELECT` on the objects they will access.


In [ ]:
for group in [GROUP_PHI, GROUP_DEIDENTIFIED]:
    spark.sql(f"GRANT USE CATALOG ON CATALOG {CATALOG} TO `{group}`;")
    spark.sql(f"GRANT USE SCHEMA ON SCHEMA {CATALOG}.{SCHEMA} TO `{group}`;")
    spark.sql(f"GRANT READ VOLUME ON VOLUME {CATALOG}.{SCHEMA}.{VOLUME} TO `{group}`;")
    spark.sql(f"GRANT SELECT ON TABLE {UC_TABLE} TO `{group}`;")
    print(f"✓ Base permissions granted to {group}")


---
## 2. Unity Catalog — Create a row-filtered dynamic view

The dynamic view uses `is_account_group_member()` to apply different `WHERE` clauses
depending on the caller's group membership:

| Group | Access |
|-------|--------|
| `pixels_phi` | **All** rows (full PHI access) |
| `pixels_deidentified` | Only rows where `DeIdentificationMethod` (DICOM tag `00120063`) is **not null** |
| Everyone else | **No** rows |


In [ ]:
ROW_FILTERED_VIEW = "dicom_row_filtered"

spark.sql(f"""
CREATE OR REPLACE VIEW {CATALOG}.{SCHEMA}.{ROW_FILTERED_VIEW} AS
SELECT * FROM {UC_TABLE}
WHERE
  CASE
    WHEN is_account_group_member('{GROUP_PHI}')          THEN TRUE
    WHEN is_account_group_member('{GROUP_DEIDENTIFIED}') THEN meta:['00120063'].Value[0] IS NOT NULL
    ELSE FALSE
  END
""")

spark.sql(f"ALTER VIEW {CATALOG}.{SCHEMA}.{ROW_FILTERED_VIEW} SET TAGS ('phi', 'pixels_anonym')")

# Grant SELECT on the view so each group can query it
for group in [GROUP_PHI, GROUP_DEIDENTIFIED]:
    spark.sql(f"GRANT SELECT ON TABLE {CATALOG}.{SCHEMA}.{ROW_FILTERED_VIEW} TO `{group}`;")

print(f"✓ Row-filtered view created: {CATALOG}.{SCHEMA}.{ROW_FILTERED_VIEW}")


### Quick validation

Query the view to see which rows the *current* user can access.


In [ ]:
df = spark.sql(f"SELECT * EXCEPT (thumbnail) FROM {CATALOG}.{SCHEMA}.{ROW_FILTERED_VIEW}")
print(f"Rows visible to current user: {df.count()}")
df.display()


---
## 3. (Optional) Column masking via dynamic view

In addition to row filtering, you can mask sensitive columns for non-PHI groups.


In [ ]:
COLUMN_MASKED_VIEW = "dicom_column_masked"

spark.sql(f"""
CREATE OR REPLACE VIEW {CATALOG}.{SCHEMA}.{COLUMN_MASKED_VIEW} AS
SELECT * EXCEPT (meta),
  CASE
    WHEN IS_ACCOUNT_GROUP_MEMBER('{GROUP_PHI}') THEN meta
    ELSE '[REDACTED]'
  END AS meta
FROM {UC_TABLE}
""")

spark.sql(f"ALTER VIEW {CATALOG}.{SCHEMA}.{COLUMN_MASKED_VIEW} SET TAGS ('phi', 'pixels_anonym')")

for group in [GROUP_PHI, GROUP_DEIDENTIFIED]:
    spark.sql(f"GRANT SELECT ON TABLE {CATALOG}.{SCHEMA}.{COLUMN_MASKED_VIEW} TO `{group}`;")

print(f"✓ Column-masked view created: {CATALOG}.{SCHEMA}.{COLUMN_MASKED_VIEW}")


---
## 4. Lakebase — Initialize the RLS schema

Connect to Lakebase and apply the RLS DDL. This creates:

| Object | Purpose |
|--------|---------|
| `pixels.access_rules` | Admin-managed table mirroring UC row filter logic |
| `pixels.user_groups` | User email → group mappings (synced from SCIM) |
| `allowed_groups TEXT[]` column | Added to `instance_paths` and `dicom_frames` |
| RLS policies | `SELECT` restricted by group overlap; `INSERT`/`UPDATE` unrestricted |


In [ ]:
from pathlib import Path
from dbx.pixels.lakebase import LakebaseUtils

# Connect to (or create) the Lakebase instance.
# Passing uc_table_name aligns the Lakebase database + schema to UC:
# catalog.schema.table → database = "catalog", schema = "schema".
lb = LakebaseUtils(
    instance_name=LAKEBASE_INSTANCE,
    create_instance=True,
    uc_table_name=UC_TABLE,
)
print(f"✓ Connected to Lakebase instance: {lb.instance_name}, database: {lb.database}, schema: {lb.schema}")


In [ ]:
import dbx.pixels.lakebase as _lb_mod

_sql_dir = Path(_lb_mod.__file__).parent / "resources" / "sql" / "lakebase"

# Apply all schema files in order.
# SQL files use {schema_name} placeholders that are filled with the UC-aligned schema.
sql_files = [
    "CREATE_LAKEBASE_SCHEMA.sql",
    "CREATE_LAKEBASE_DICOM_FRAMES.sql",
    "CREATE_LAKEBASE_RLS.sql",               # ← RLS tables, columns & policies
]

for sql_file in sql_files:
    with open(_sql_dir / sql_file) as fh:
        lb.execute_query(fh.read().format(schema_name=lb.schema))
    print(f"  ✓ Applied {sql_file}")

print(f"\n✓ Lakebase schema '{lb.schema}' initialised with RLS support")


---
## 5. Sync user → group mappings from Databricks SCIM

This pulls every user and their group memberships from the workspace SCIM API and
upserts them into `pixels.user_groups`. The DICOMweb app reads this table at request
time to resolve the caller's groups.

> **Tip**: Schedule this cell as a periodic job (e.g. daily) to keep mappings current.


In [ ]:
synced_count = lb.sync_user_groups_from_databricks()
print(f"✓ Synced {synced_count} (user, group) pairs from SCIM to Lakebase")


In [ ]:
# Verify: show the first 20 user-group mappings
from psycopg2 import sql as psql
rows = lb.execute_and_fetch_query(
    psql.SQL("SELECT user_email, group_name, synced_at FROM {} ORDER BY user_email LIMIT 20").format(
        psql.Identifier(lb.schema, "user_groups")
    )
)
print(f"{'Email':<45} {'Group':<30} {'Synced At'}")
print("-" * 95)
for email, group, synced_at in rows:
    print(f"{email:<45} {group:<30} {synced_at}")


---
## 6. Define access rules in Lakebase

Access rules mirror the UC row filter logic. Each rule maps a `(uc_table, group)` pair to an access pattern:

| `access_type` | Meaning |
|---------------|---------|
| `full` | The group may see **all** rows cached from this UC table |
| `conditional` | The group may only see rows matching `uc_filter_sql` |

The `uc_filter_sql` column must contain a valid Unity Catalog `WHERE` clause. The sync job (Step 7) will execute it against the UC table to discover which SOP Instance UIDs match.


In [ ]:
# Rule 1: pixels_phi → full access to all rows
lb.upsert_access_rule(
    uc_table_name=UC_TABLE,
    group_name=GROUP_PHI,
    access_type="full",
    description="Full access to all DICOM data (PHI group)",
)

# Rule 2: pixels_deidentified → only de-identified data
lb.upsert_access_rule(
    uc_table_name=UC_TABLE,
    group_name=GROUP_DEIDENTIFIED,
    access_type="conditional",
    uc_filter_sql="meta:['00120063'].Value[0] IS NOT NULL",
    description="Access only to de-identified DICOM data",
)

print("✓ Access rules upserted")


In [ ]:
# Verify: list all rules for this table
rules = lb.get_access_rules(UC_TABLE)
print(f"Access rules for {UC_TABLE}:\n")
for r in rules:
    print(f"  Group:       {r['group_name']}")
    print(f"  Access type: {r['access_type']}")
    print(f"  Filter SQL:  {r['uc_filter_sql'] or '(none — full access)'}")
    print(f"  Description: {r['description']}")
    print()


---
## 7. Sync UC row filters → Lakebase `allowed_groups`

This is the **core sync step**. For each access rule:

- **`full`** rules → the group is added to `allowed_groups` on *every* cached row for the table.
- **`conditional`** rules → the sync job queries the UC table with the `uc_filter_sql` to find matching SOP Instance UIDs, then adds the group only to *those* cached rows.

The sync is **additive** (groups are appended, never removed). To do a clean re-sync, call `reset_allowed_groups()` first.

> **Note**: The SQL client must authenticate as a **service principal** with full `SELECT` on the UC table (not subject to the row filter). This ensures the conditional queries can see all rows.


In [ ]:
import os
from databricks.sdk.core import Config
from dbx.pixels.resources.dicom_web.utils.sql_client import DatabricksSQLClient

cfg = Config()
sql_client = DatabricksSQLClient(
    host=cfg.host,
    warehouse_id=os.environ["DATABRICKS_WAREHOUSE_ID"],
)

print(f"✓ SQL client connected to warehouse {os.environ['DATABRICKS_WAREHOUSE_ID']}")


In [ ]:
# Optional: reset allowed_groups before a full re-sync
# Uncomment the next line for a clean slate (all cached rows become unrestricted
# until the sync completes).

# lb.reset_allowed_groups(UC_TABLE)
# print(f"✓ Reset allowed_groups for {UC_TABLE}")


In [ ]:
updated = lb.sync_uc_row_filters(UC_TABLE, sql_client)
print(f"\n✓ Sync complete — {updated} Lakebase row(s) updated")


---
## 8. Verify RLS enforcement in Lakebase

Let's simulate what different users would see when querying Lakebase directly. We use `SET LOCAL app.user_groups` to set the session context — this is exactly what the DICOMweb application does before each Lakebase query.


In [ ]:
def count_visible_rows(lb_utils, table_name: str, user_groups: list[str]) -> int:
    """Count how many rows are visible under the given user groups."""
    from psycopg2 import sql as psql
    rows = lb_utils.execute_and_fetch_query(
        psql.SQL("SELECT COUNT(*) FROM {} WHERE uc_table_name = %s").format(
            psql.Identifier(lb_utils.schema, table_name)
        ),
        params=(UC_TABLE,),
        user_groups=user_groups,
    )
    return rows[0][0] if rows else 0


# Rows visible to PHI group (should be ALL)
phi_ip = count_visible_rows(lb, "instance_paths", [GROUP_PHI])
phi_df = count_visible_rows(lb, "dicom_frames", [GROUP_PHI])

# Rows visible to de-identified group (should be a subset)
deid_ip = count_visible_rows(lb, "instance_paths", [GROUP_DEIDENTIFIED])
deid_df = count_visible_rows(lb, "dicom_frames", [GROUP_DEIDENTIFIED])

# Rows visible with no groups (should be only unrestricted/legacy rows)
anon_ip = count_visible_rows(lb, "instance_paths", [])
anon_df = count_visible_rows(lb, "dicom_frames", [])

print(f"{'Group':<30} {'instance_paths':>16} {'dicom_frames':>16}")
print("-" * 65)
print(f"{GROUP_PHI:<30} {phi_ip:>16,} {phi_df:>16,}")
print(f"{GROUP_DEIDENTIFIED:<30} {deid_ip:>16,} {deid_df:>16,}")
print(f"{'(no groups)':<30} {anon_ip:>16,} {anon_df:>16,}")


---
## 9. Enable RLS on the DICOMweb App

To activate RLS enforcement at runtime, set these environment variables on your Databricks App deployment:

```bash
# Enable on-behalf-of (OBO) user authentication
export DICOMWEB_USE_USER_AUTH=true

# Enable Lakebase Row-Level Security
export LAKEBASE_RLS_ENABLED=true

# Initialize Lakebase schema on startup (creates RLS tables + policies)
export LAKEBASE_INIT_DB=true

# Lakebase instance name
export LAKEBASE_INSTANCE_NAME=pixels-lakebase
```

Once active, the application will:

1. Resolve the user's email from the `X-Forwarded-Email` header.
2. Look up their groups from `pixels.user_groups` in Lakebase.
3. Set `SET LOCAL app.user_groups = '...'` on every Lakebase query.
4. Include a groups hash in the in-memory cache key.

### Security matrix

| Cache Tier | Protection Mechanism |
|------------|---------------------|
| Tier 1 — In-memory | Cache keys scoped by groups hash |
| Tier 2 — Lakebase | PostgreSQL RLS with `allowed_groups && user_groups` |
| Tier 3 — UC SQL Warehouse | Native UC row filters (OBO token) |
| File reads (Volumes API) | OBO token → UC ACLs at storage layer |


---
## 10. Ongoing maintenance

Schedule the following as periodic jobs to keep the RLS mappings current:

```python
from dbx.pixels.lakebase import LakebaseUtils
from dbx.pixels.resources.dicom_web.utils.sql_client import DatabricksSQLClient

lb = LakebaseUtils(instance_name="pixels-lakebase")

# 1. Re-sync user → group mappings (new hires, role changes)
lb.sync_user_groups_from_databricks()

# 2. Re-sync UC row filters → Lakebase allowed_groups
#    (after adding new access rules or when cached data changes)
sql_client = DatabricksSQLClient(host="...", warehouse_id="...")
lb.reset_allowed_groups("main.pixels_solacc.object_catalog")
lb.sync_uc_row_filters("main.pixels_solacc.object_catalog", sql_client)
```

### Adding a new access group

1. Create the group in Databricks Account Console.
2. Add the group to Unity Catalog row filter logic (update the dynamic view).
3. Add a matching access rule in Lakebase:

```python
lb.upsert_access_rule(
    uc_table_name="main.pixels_solacc.object_catalog",
    group_name="new_group_name",
    access_type="conditional",
    uc_filter_sql="meta:['00120063'].Value[0] = 'Per DICOM PS 3.15 AnnexE'",
    description="Access to data de-identified with a specific method",
)
```

4. Re-run the sync:

```python
lb.sync_user_groups_from_databricks()
lb.reset_allowed_groups("main.pixels_solacc.object_catalog")
lb.sync_uc_row_filters("main.pixels_solacc.object_catalog", sql_client)
```


---
## 6. Define access rules in Lakebase

Access rules mirror the UC row filter logic. Each rule maps a `(uc_table, group)` pair
to an access pattern:

| `access_type` | Meaning |
|---------------|---------|
| `full` | The group may see **all** rows cached from this UC table |
| `conditional` | The group may only see rows matching `uc_filter_sql` |

The `uc_filter_sql` column must contain a valid **Unity Catalog** `WHERE` clause.
The sync job (Step 7) will execute it against the UC table to discover which
SOP Instance UIDs match.
